In [3]:
# For an elliptic curve, with notation as in the paper,
# this block of code computes the preimage of a divisor 
# under the map Frac(\overline{R}) \rightarrow \Div^0(\overline}{E})
# if such a preimage exists.

def lineBetweenPoints(P, Q):
    a1, b1 = P.xy()
    a2, b2 = Q.xy()
    if (a1 == a2):
        return (x-a1)
    return ((b2-b1)/(a2-a1))*(x-a1) - (y-b1)

def lineTangentTo(P, E):
    a, b = P.xy()
    if (P.order() == 2):
        return (x-a)
    defining_array = E.a_invariants()
    a1 = defining_array[0]
    a2 = defining_array[1]
    a3 = defining_array[2]
    a4 = defining_array[3]
    a6 = defining_array[4]
    slope = (3*a^2 + 2*a2*a - a1*b + a4)/(2*b + a1*a+a3)
    return slope*(x-a) - (y-b)

class Divisor:
    def __init__(self, E, dict, base_point):
        self.E = E
        self.base_point = base_point
        self.dict = {}
        for key, value in dict.items():
            if value != 0:
                self.dict[key] = value

    def __add__(self, another_divisor):
        new_dict = {}
        for key, value in self.dict.items():
            if value is not 0:
                new_dict[key] = value

        for key, value in another_divisor.dict.items():
            if key in new_dict.keys() and new_dict[key] + value is not 0:
                new_dict[key] = new_dict[key] + value
            elif value is not 0:
                new_dict[key] = value
        return Divisor(self.E, new_dict, base_point)

    def __sub__(self, another_divisor):
        new_dict = {}
        for key, value in self.dict.items():
            if value is not 0:
                new_dict[key] = value

        for key, value in another_divisor.dict.items():
            if key in new_dict.keys() and new_dict[key] - value is not 0:
                new_dict[key] = new_dict[key] - value
            elif value is not 0:
                new_dict[key] = -value
        return Divisor(self.E, new_dict, base_point)

    def __eq__(self, another_divisor):
        if (self.E is not another_divisor.E):
            return False
        elif self.dict == another_divisor.dict:
            return True
        else:
            return False

    def __ne__(self, another_divisor):
        return not (self == another_divisor)

    def coeff(self, point):
        return self.dict[point]

    def support(self):
        return self.dict.keys()

    def getMNPQ(self):
        m = 0
        n = 0
        P = None
        Q = None
        for point in self.support():
            if (self.coeff(point) > 0 and point != base_point):
                P = point
                m = self.coeff(point)
            elif (self.coeff(point) < 0 and point != base_point):
                Q = point
                n = -self.coeff(point)
        return m, n, P, Q

    def getPair(self, comparisonOperator, equalityOperator):
        points = set()
        for point in self.support():
            if point != base_point and (comparisonOperator(self.coeff(point), 0)):
                points.add(point)
        for P in points:
            for Q in points:
                if (P != Q and equalityOperator(P,-Q)):
                    return (P,Q)

    def existsPair(self, comparisonOperator, equalityOperator):
        points = set()
        for point in self.support():
            if point != base_point and (comparisonOperator(self.coeff(point), 0)):
                points.add(point)
        for P in points:
            for Q in points:
                if (P != Q and equalityOperator(P,-Q)):
                    return True
        return False

def interpolate(D, base_point, E):
    # start with rational function 1
    # while D is not 0
    rational_function = 1
    E = D.E
    zero_divisor = Divisor(E,{},base_point)
    while (D != zero_divisor):
        #print "loop"
        #print D.dict
        #print rational_function
        # if P, Q have coefficients > 0 and P != -Q, subtract (<P>  + <Q> + <-(P+Q)> - 3<0>) and multiply rational function by line between P and Q
        if D.existsPair(operator.gt, operator.ne):
            #print "case 1"
            P, Q = D.getPair(operator.gt, operator.ne)
            sub_divisor = Divisor(E, {P : 1, Q : 1, -(P+Q) : 1, base_point: -3}, base_point)
            D = D - sub_divisor
            rational_function = rational_function*lineBetweenPoints(P,Q)
        # else if P, Q have coefficients > 0 and P=-Q subtract (<P> + <Q> - 2<0>) and multiply rational function by line between P and Q
        elif D.existsPair(operator.gt, operator.eq):
            #print "case 2"
            P, Q = D.getPair(operator.gt, operator.eq)
            sub_divisor = Divisor(E, {P:1,Q:1, base_point: -2}, base_point)
            D = D - sub_divisor
            rational_function = rational_function*lineBetweenPoints(P,Q)
        # else if P, Q have coefficients < 0 and P != -Q, add (<P>  + <Q> + <-(P+Q)> - 3<0>) and divide rational function by line between P and Q
        elif D.existsPair(operator.lt, operator.ne):
            #print "case 3"
            P, Q = D.getPair(operator.lt, operator.ne)
            add_divisor = Divisor(E, {P:1,Q:1,-(P+Q):1,base_point:-3}, base_point)
            D = D+add_divisor
            rational_function = rational_function/lineBetweenPoints(P,Q)
        # else if P, Q have coefficients < 0 and P=-Q add (<P> + <Q> - 2<0>) and divide rational function by line between P and Q
        elif D.existsPair(operator.lt, operator.eq):
            #print "case 4"
            P, Q = D.getPair(operator.lt, operator.eq)
            add_divisor = Divisor(E, {P:1,Q:1,base_point:-2}, base_point)
            D = D+add_divisor
            rational_function = rational_function/lineBetweenPoints(P,Q)
        # else if D is of form m<P> - n<Q> + o<0> (o may be positive or negative, m >= 0, n>= 0)
        else:
            m, n, P, Q = D.getMNPQ()
            # if m >= 2 and P not order 2, then subtract (2<P> + <-2P> - 3<0>) and multiply rational function by tangent line at P
            if m >= 2 and P.order() != 2:
                #print "case 5"
                if P.order() == 3:
                    # -2P = P
                    sub_divisor = Divisor(E, {P:3, base_point:-3}, base_point)
                else:
                    sub_divisor = Divisor(E, {P:2, (-2*P):1, base_point:-3}, base_point)
                D = D - sub_divisor
                rational_function = rational_function*lineTangentTo(P, E)
            # else if m >= 2 and P of order 2, then subtract (2<P> - 2<0>) and multiply rational function by tangent line at P
            elif m >= 2 and P.order() == 2:
                #print "case 6"
                sub_divisor = Divisor(E, {P:2,base_point:-2}, base_point)
                D = D - sub_divisor
                rational_function = rational_function*lineTangentTo(P, E)
            # else if n >=2 and Q not of order 2, then add (2<Q> + <-2Q> - 3<0>) and divide rational function by tangent line at Q
            elif n >=2 and Q.order() != 2:
                #print "case 7"
                if Q.order() == 3:
                    # -2Q = Q
                    add_divisor = Divisor(E, {Q:3, base_point: -3}, base_point)
                else:
                    add_divisor = Divisor(E, {Q:2, (-2*Q):1, base_point: -3}, base_point)
                D = D + add_divisor
                rational_function = rational_function/lineTangentTo(Q, E)
            # else if n >= 2 and Q of order 2, then add (2<Q> - 2<0>) and divide rational function by tangent line at Q
            elif n >=2 and Q.order() == 2:
                #print "case 8"
                add_divisor = Divisor(E, {Q:2, base_point:-2}, base_point)
                D = D + add_divisor
                rational_function = rational_function/lineTangentTo(Q, E)
            # else if m=n=1, add (<Q> + <-Q> - 2<0>) and divide rational function by line through Q and -Q
            elif m==1 and n==1:
                #print "case 9"
                add_divisor = Divisor(E, {Q:1, -Q:1, base_point:-2}, base_point)
                D = D + add_divisor
                rational_function = rational_function/lineBetweenPoints(Q,-Q)
            else:
                print "cannot interpolate the following divisor"
                print D.dict
                print rational_function
                return None
    return rational_function

In [ ]:
# Example 1: computing the units of the Fermat 3-curve
from sage.modules.free_module_integer import IntegerLattice
from itertools import product

# construct the curve
a = PolynomialRing(RationalField(), 'a').gen()
K.<b> = NumberField(a^2 + a + 1)
R.<x,y,z> = PolynomialRing(K,3)
mor = EllipticCurve_from_cubic(x^3+y^3-z^3, [1,-1,0], morphism=True)
E = EllipticCurve_from_cubic(x^3+y^3-z^3, [1,-1,0], morphism=False)

# construct the boundary points

T1 = E(mor([0,1,1]))
T2 = E(mor([0,b,1]))
T3 = E(mor([0,b^2,1]))
T4 = E(mor([1,0,1]))
T5 = E(mor([b,0,1]))
T6 = E(mor([b^2,0,1]))
T7 = E(mor([-1,1,0]))
T8 = E(mor([-b,1,0]))
T9 = E(mor([-b^2,1,0]))

# ommitted -- we compute the lattice of relations, and get 
# the following matrix using the algorithm described
# in the paper

#[ 1  0  0  0  0  1  0  0 -2] 1
#[ 0  1  0  0  0  1  0  1 -3] 2
#[ 0  0  1  0  0  1  0  2 -4] 3
#[ 0  0  0  1  0  2  0  2 -5] 4
#[ 0  0  0  0  1  2  0  1 -4] 5
#[ 0  0  0  0  0  3  0  0 -3] 6
#[ 0  0  0  0  0  0  1  1 -2] 7
#[ 0  0  0  0  0  0  0  3 -3] 8

base_point = E([0,1,0])

#interpolate each divisor
print interpolate(Divisor(E, {T1:1, T6:1, T9:-2}, base_point), base_point, E)
print interpolate(Divisor(E, {T2:1, T6:1, T8:1, T9:-3}, base_point), base_point, E)
print interpolate(Divisor(E, {T3:1, T6:1, T8:2, T9:-4}, base_point), base_point, E)
print interpolate(Divisor(E, {T4:1, T6:2, T8:2, T9:-5}, base_point), base_point, E)
print interpolate(Divisor(E, {T5:1, T6:2, T8:1, T9:-4}, base_point), base_point, E)
print interpolate(Divisor(E, {T6:3, T9:-3}, base_point), base_point, E)
print interpolate(Divisor(E, {T7:1, T8:1, T9:-2}, base_point), base_point, E)
print interpolate(Divisor(E, {T8:3, T9:-3}, base_point), base_point, E)

In [4]:
# Example 2: computing the units of an elliptic curve
# with defining equation y^2 = x^3 - x + root(2)
from sage.modules.free_module_integer import IntegerLattice
from itertools import product

# construct an appropriate number field
a = PolynomialRing(RationalField(), 'a').gen()
M.<preroot4of2> = NumberField(a^4 - 2)
L.<prei> = M.extension(a^2 + 1)
K.<d> = L.absolute_field()

# note that taking the absolute_field creates an injection of fields
# so we are now working in K
fm = K.structure()[1]
i = fm(prei)
root4of2 = fm(preroot4of2)
root2 = root4of2^2

# construct the elliptic curve
R.<x,y,z> = PolynomialRing(K,3)
E = EllipticCurve([0,0,0,-1, root2])

# construct the points
T1 = E([0,1,0])
T2 = E([-root2,0,1])
T3 = E([(1+i)/root2,0,1])
T4 = E([(1-i)/root2,0,1])

Q1 = E([0,root4of2,1])
Q2 = E([0,-root4of2,1])

# we compute that
# T1.order() = 1
# T2.order() = 2
# T3.order() = 2
# T4.order() = 2
# Q1.order() = infinity
# Q2.order() = infinity

print E.height_pairing_matrix([Q1,Q2])

# the matrix will be
# [ 0.570850711738291 -0.570850711738291]
# [-0.570850711738291  0.570850711738291]
# which indicates the relation Q1 + Q2 = 0

[ 0.570850711738291 -0.570850711738291]
[-0.570850711738291  0.570850711738291]


In [2]:
# Example 2 continued 

# from before we have Q1 + Q2 = 0 
# the only nontrivial relation between the torsion points are:
# T2 + T3 + T4 = 0
# T1 = 0
# 2*T2 = 0
# 2*T3 = 0
# 2*T4 = 0


L = IntegerLattice([[1,0,0,0,0,0],
                    [0,2,0,0,0,0],
                    [0,0,2,0,0,0],
                    [0,0,0,2,0,0],
                    [0,1,1,1,0,0],
                    [0,0,0,0,1,1]])

H = IntegerLattice([[1,-1,0,0,0,0],
                    [0,1,-1,0,0,0],
                    [0,0,1,-1,0,0],
                    [0,0,0,1,-1,0],
                    [0,0,0,0,1,-1]])

# print our basis of relations
print H.intersection(L).basis()



Free module of degree 6 and rank 4 over Integer Ring
Echelon basis matrix:
[ 1  1  1  1 -2 -2]
[ 0  2  0  0 -1 -1]
[ 0  0  2  0 -1 -1]
[ 0  0  0  2 -1 -1]
[
(1, 1, 1, 1, -2, -2),
(0, 2, 0, 0, -1, -1),
(0, 0, 2, 0, -1, -1),
(0, 0, 0, 2, -1, -1)
]


In [5]:
# Example 2 continued
# interpolate
# [ 1  1  1  1 -2 -2]
# [ 0  2  0  0 -1 -1]
# [ 0  0  2  0 -1 -1]
# [ 0  0  0  2 -1 -1]

base_point = E([0,1,0])
print interpolate(Divisor(E, {T1:1, T2:1, T3:1, T4:1, Q1:-2, Q2:-2}, base_point), base_point, E)
print interpolate(Divisor(E, {T2:2, Q1:-1, Q2:-1}, base_point), base_point, E)
print interpolate(Divisor(E, {T3:2, Q1:-1, Q2:-1}, base_point), base_point, E)
print interpolate(Divisor(E, {T4:2, Q1:-1, Q2:-1}, base_point), base_point, E)

(-y)/x^2
(x + (-1/12*d^6 - 5/12*d^4 - 1/12*d^2 - 17/12))/x
(x + (-1/8*d^7 + 1/24*d^6 - 1/2*d^5 + 5/24*d^4 - 3/8*d^3 + 1/24*d^2 - 15/4*d + 17/24))/x
(x + (1/8*d^7 + 1/24*d^6 + 1/2*d^5 + 5/24*d^4 + 3/8*d^3 + 1/24*d^2 + 15/4*d + 17/24))/x


In [6]:
# Example 2 continued
# translate the results back from coefficients in K 
# to coefficients in L to get our final unit group generators
revfm = K.structure()[0]
print revfm(-1/12*d^6 - 5/12*d^4 - 1/12*d^2 - 17/12)
print revfm(-1/8*d^7 + 1/24*d^6 - 1/2*d^5 + 5/24*d^4 - 3/8*d^3 + 1/24*d^2 - 15/4*d + 17/24)
print revfm(1/8*d^7 + 1/24*d^6 + 1/2*d^5 + 5/24*d^4 + 3/8*d^3 + 1/24*d^2 + 15/4*d + 17/24)

preroot4of2^2
-1/2*preroot4of2^2*prei - 1/2*preroot4of2^2
1/2*preroot4of2^2*prei - 1/2*preroot4of2^2


In [11]:
# Example 3: computing units of an elliptic curve
# defined by equation y^2 = (x-1)(x+1)(x-4)

# construct the curve
E = EllipticCurve([0,-4,0,-1,4])
# note that the j-invariant is negative under the 5-adic valuation and
# thus the intrinsic tropicalization will be interesting
print E.j_invariant()
# construct the poitns
T1 = E([0,1,0])
T2 = E([1,0,1])
T3 = E([-1,0,1])
T4 = E([4,0,1])

Q1 = E([0,2,1])
Q2 = E([0,-2,1])

# compute their orders
print "\ntorsion orders:"
print T1.order()
print T2.order()
print T3.order()
print T4.order()
print Q1.order()
print Q2.order()

# the lattice of relations is
# interpolate
# [ 1  1  1  1 -2 -2]
# [ 0  2  0  0 -1 -1]
# [ 0  0  2  0 -1 -1]
# [ 0  0  0  2 -1 -1]

base_point = E([0,1,0])
R.<x,y,z> = PolynomialRing(QQ,3)

# interpolate our divisors
print "\nthe final units:"
print interpolate(Divisor(E, {T1:1, T2:1, T3:1, T4:1, Q1:-2, Q2:-2}, base_point), base_point, E)
print interpolate(Divisor(E, {T2:2, Q1:-1, Q2:-1}, base_point), base_point, E)
print interpolate(Divisor(E, {T3:2, Q1:-1, Q2:-1}, base_point), base_point, E)
print interpolate(Divisor(E, {T4:2, Q1:-1, Q2:-1}, base_point), base_point, E)

438976/225

torsion orders:
1
2
2
2
+Infinity
+Infinity

the final units:
(-y)/x^2
(x - 1)/x
(x + 1)/x
(x - 4)/x
